In [ ]:
#r "nuget: libtorch-cpu, 1.8.0.7"
#r "nuget: TorchSharp, 0.91.52518"

open TorchSharp
open TorchSharp.Tensor
open TorchSharp.NN
let device = Torch.InitializeDevice Device.CPU

Installed package TorchSharp version 0.91.52518

Installed package libtorch-cpu version 1.8.0.7

TorchSharp: LoadNativeBackend: Native backend not found in application loading TorchSharp directly from packages directory.


TorchSharp: LoadNativeBackend: Trying dynamic load for .NET/F# Interactive by consolidating native libtorch-cpu-* binaries to /home/peter/.nuget/packages/torchsharp/0.91.52518/lib/netcoreapp3.1/cpu...


In [ ]:
(*
#### PCA + Deconvolution

We render 1000s of traces with 1D random `xs` and calculated `ys` and wish to infer parameters of transform.

The transform is done by:
- choosing a random mixture of 4 component functions (e.g. sin(x), cos(x))
- adding the mixture into a kernel
- use the kernel to convolve random vector of `xs` into `ys`

The task is to reconstruct the:
- shape of 4 components inside kernels
- mixture weights for 4 components for each sample (embedding lookups)

*)

In [ ]:
// helper
type TorchTensor with
    member t.toArray() =
        match t.shape with
        | [|n|] -> Array.init (int n) (fun i -> t.[int64 i].ToSingle())
        | _ -> failwithf "requires 1-dimensional tensor, got %A" t.shape

In [ ]:
let xsLen = 32
let convLen = 16
let components = [| // component functions that we will later try to reconstruct (deconvolve)
    fun x -> (Math.Sin (x/float convLen*3.14) * 0.5 + 0.5)
    fun x -> (Math.Cos (x/float convLen*3.14) * 0.5 + 0.5)
    fun x -> (-Math.Sin (x/float convLen*3.14) * 0.5 + 0.5)
    fun x -> (-Math.Cos (x/float convLen*3.14) * 0.5 + 0.5)
|]
let nComponents = components.Length

let random = Random()
module SampleGenerator =
    let dirichlet n =
        let ps = Array.init n (fun _ -> random.NextDouble()**2.0)
        let sum = ps |> Array.sum
        ps |> Array.map (fun p -> p / sum)
    let randomComponentMixtureKernel len =
        let mixture = dirichlet components.Length
        let compMix = Array.zip components mixture
        Array.init len (fun i -> compMix |> Array.sumBy (fun (compFn,weight) -> (compFn (float i) * weight)))
    let conv (kernel: float[]) (xs: float[]) =
        Array.init (xs.Length-kernel.Length+1) (fun i ->
            let mutable sum = 0.0
            for j in 0 .. kernel.Length-1 do sum <- sum + xs.[i+j] * kernel.[j]
            sum)
    let flipCoin (rate: float) (trials: float) = Math.Round(trials * rate)
    let renderSample xsLen convLen =
        let kernel = randomComponentMixtureKernel convLen
        let xs = Array.init xsLen (fun _ -> random.Next 1000 |> float)
        let rate = random.NextDouble() * 0.01
        let ys = xs |> conv kernel |> Array.map (flipCoin rate)
        xs, ys

SampleGenerator.renderSample xsLen convLen

Item1,Item2
"[ 650, 204, 361, 559, 595, 292, 690, 216, 763, 812, 345, 272, 550, 628, 35, 450, 261, 427, 56, 295 ... (12 more) ]","[ 17, 16, 17, 16, 16, 15, 15, 14, 15, 14, 14, 15, 16, 16, 17, 19, 19 ]"


In [ ]:
let nTraces = 10000

let renderDataset xsLen convLen nTraces =
    let tensor (xs: float[]) = Float32Tensor.from(Array.map float32 xs, false)
    let xs, ys =
        Array.init nTraces (fun _ -> 
            let xs, ys = SampleGenerator.renderSample xsLen convLen
            tensor xs, tensor ys)
        |> Array.unzip
    let mutable i0=0
    fun batchSize ->
        let xs, ys, indices =
            Array.init batchSize (fun i -> 
                i0 <- (i0+1) % nTraces
                xs.[i0], ys.[i0], i0)
            |> Array.unzip3
        xs.stack 0L, ys.stack 0L, Int32Tensor.from(indices, false)

let generateBatch = renderDataset xsLen convLen nTraces

In [ ]:
let inline fscalar x = TorchScalar.op_Implicit (float32 x)
let epsilon = fscalar 10e-12

// this model works fine

type Model(device, nTraces, convLen, nComponents) =
    inherit CustomModule("deconv")
    let nTraces, nComponents, convLen = int64 nTraces, int64 nComponents, int64 convLen
    let logKernel = Float32Tensor.rand([|nComponents; 1L; convLen|], device, true)
    let logScale = Float32Tensor.from([|-7.0f|], true)
    let logEmbeddings = TorchSharp.NN.Modules.Embedding( nTraces, nComponents )

    member this.parameters = [| logKernel; logScale; logEmbeddings.Weight |]

    override _.forward (x:TorchTensor) = failwithf "wrong method"

    member n.forward (xs:TorchTensor, indices:TorchTensor) =
        let factors = (logEmbeddings.forward indices + logScale).exp().unsqueeze(2L)
        let kernel = logKernel.exp()
        let ins = xs.unsqueeze(1L).expand([|-1L; nComponents; -1L;|])
        //printfn "ins=%A kernel=%A" ins.shape kernel.shape
        let compOuts = ins.conv1d(kernel, groups=nComponents)
        //printfn "conv=%A, factors=%A, kernel=%A" compOuts.shape factors.shape kernel.shape
        let outs = (compOuts * factors).sum([|1L|], keepDimension=false) + epsilon //* globalFactors.exp()
        outs

    member _.Kernel with get () = logKernel.exp()
    member _.Scale with get () = logScale
    member n.modelLoss() =
        (logKernel.exp().sum([|2L|], keepDimension=true) - fscalar (convLen/2L)).abs().mean()
        //+ (logEmbeddings.Weight.exp().sum([|1L|], keepDimension=true) - fscalar 1.0).mean()

let net = new Model(device, nTraces, convLen, nComponents)

In [ ]:
// this model leaks memory

type Model(device, nTraces, convLen, nComponents) =
    inherit CustomModule("deconv")
    let nTraces, nComponents, convLen = int64 nTraces, int64 nComponents, int64 convLen
    member _.logKernel = Float32Tensor.zeros([|nComponents; 1L; convLen|], device, true)
    member _.logScale = Float32Tensor.from([|-7.0f|], true)
    member _.logEmbeddings = TorchSharp.NN.Modules.Embedding( nTraces, nComponents )

    member n.parameters = [| n.logKernel; n.logScale; n.logEmbeddings.Weight |]

    override _.forward (x:TorchTensor) = failwithf "wrong method"

    member n.forward (xs:TorchTensor, indices:TorchTensor) =
        let factors = (n.logEmbeddings.forward indices + n.logScale).exp().unsqueeze(2L)
        let kernel = n.logKernel.exp()
        let ins = xs.unsqueeze(1L).expand([|-1L; nComponents; -1L;|])
        //printfn "ins=%A kernel=%A" ins.shape kernel.shape
        let compOuts = ins.conv1d(kernel, groups=nComponents)
        //printfn "conv=%A, factors=%A, kernel=%A" compOuts.shape factors.shape kernel.shape
        let outs = (compOuts * factors).sum([|1L|], keepDimension=false) + epsilon //* globalFactors.exp()
        outs

    member n.Kernel with get () = n.logKernel.exp()
    member n.Scale with get () = n.logScale
    member n.modelLoss() =
        (n.logKernel.exp().sum([|2L|], keepDimension=true) - fscalar (convLen/2L)).abs().mean()


let net = new Model(device, nTraces, convLen, nComponents)

In [ ]:
let xs, ys, indices = generateBatch 5
let ys' = net.forward(xs,indices)
//sprintf "shapes: xs=%A ys=%A indices=%A -> ys'=%A" xs.shape ys.shape indices.shape ys'.shape
sprintf "%A -> %A" (ys.[0L].toArray()) (ys'.[0L].toArray())

[|15.0f; 14.0f; 14.0f; 15.0f; 16.0f; 16.0f; 16.0f; 18.0f; 16.0f; 16.0f; 16.0f;
  16.0f; 17.0f; 18.0f; 17.0f; 15.0f; 15.0f|] -> [|37.0091362f; 33.872509f; 30.73075294f; 34.47790527f; 36.34897614f;
  35.56280518f; 36.10554504f; 39.22198486f; 38.00999451f; 36.23498535f;
  38.23213577f; 34.38507843f; 37.17513275f; 40.79251862f; 38.19708252f;
  37.7118454f; 36.31793976f|]

In [ ]:
let xlogy(x:TorchTensor, y:TorchTensor) = (x.clamp_min epsilon) * y.log()

let poissonLoss (k:TorchTensor) (mu:TorchTensor) =
    let logPmf = xlogy(k,mu) - (k+fscalar 1.0).lgamma() - mu
    -logPmf

let criterion ys ys' = (poissonLoss ys ys').clamp_max(fscalar 10000.0).mean()
//let criterion (ys:TorchTensor) (ys':TorchTensor) = let d = ys - ys' in (d*d).mean()

let xs, ys, indices = generateBatch 1
let ys' = net.forward(xs,indices)
//printfn "xs.shape=%A ys'.shape=%A, result=%A" xs.shape ys'.shape (ys'.[0L].toArray())
(criterion ys ys').ToDouble()

9.107818603515625

In [ ]:
let optimizer = TorchSharp.NN.Optimizer.Adam(net.parameters, 0.02)
//let optimizer = TorchSharp.NN.Optimizer.SGD(net.parameters, 0.1)

In [ ]:
let batchSize = 384
let mutable cumLoss = 0.0
let mutable nItems = 0
for i in 0..200000 do
    net.ZeroGrad()
    optimizer.zero_grad()
    let xs, ys, indices = generateBatch batchSize
    let ys' = net.forward(xs,indices)
    let loss = criterion ys ys' + net.modelLoss()
    loss.backward()
    optimizer.step()

    cumLoss <- cumLoss + loss.ToDouble()
    nItems <- nItems + 1
    if i%10000 = 0 then
        System.Console.Write $"step %6d{i}: loss=%.4f{cumLoss / float nItems} loss0=%.4f{loss.ToDouble()} scale=%.4f{net.Scale.ToDouble()}"
        cumLoss <- 0.0
        nItems <- 0
    if i%1000 = 0 then
        GC.Collect()

step      0: loss=65.5197 loss0=65.5197 scale=-7.0200

step  10000: loss=2.6470 loss0=2.3202 scale=-7.6443

step  20000: loss=2.3941 loss0=2.3346 scale=-7.7071

step  30000: loss=2.3895 loss0=2.4422 scale=-8.0270

In [ ]:
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json"
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json"

#r "nuget: Plotly.NET, 2.0.0-beta9"
#r "nuget: Plotly.NET.Interactive, 2.0.0-beta9"
open Plotly.NET


Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json

Installed package Plotly.NET version 2.0.0-beta9

Installed package Plotly.NET.Interactive version 2.0.0-beta9

Loading extensions from `Plotly.NET.Interactive.dll`

Added Kernel Extension including formatters for GenericChart

In [ ]:
let traces = [
    for i in 0L..3L ->
        let ys = net.Kernel.[i].[0L].toArray()
        let xs = [|0..ys.Length|]
        Chart.Line(xs, ys)
]
traces
|> Chart.Combine
|> Chart.withSize(1000.,600.)


<!-- Plotly chart will be drawn inside this DIV -->